In [2]:
import os
from pymilvus import MilvusClient
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import math
import base64
from time import gmtime, strftime
import random, string
import time
import base64
import uuid
import socket
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
from datetime import datetime, timedelta
from langchain_milvus import Milvus

DIMENSION = 384 
TOKEN = os.environ.get("ZILLIZ_TOKEN")
MILVUS_URL = os.environ.get("SERVERLESS_VECTORDB_URI")

slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

OAQ3_COLLECTION_NAME = "openaqmeasurements"
TODAYS_DATE = str( datetime.today().strftime('%Y-%m-%d') ) 
YESTERDAYS_DATE = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.hub.langchain.com"
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

### Turn off slack warnings
os.environ["SKIP_SLACK_SDK_WARNING"] = "false"


In [3]:
# -----------------------------------------------------------------------------
# Connect to Milvus
# https://zilliz.com/blog/a-beginners-guide-to-using-llama-3-with-ollama-milvus-langchain
# https://github.com/stephen37/ollama_local_rag/blob/main/rag_berlin_parliament.py
# https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/
# https://api.python.langchain.com/en/latest/vectorstores/langchain_milvus.vectorstores.milvus.Milvus.html

vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=OAQ3_COLLECTION_NAME,
    primary_field = "id",
    text_field="details",
    connection_args={"uri": MILVUS_URL, "token": TOKEN},
)

In [4]:
def run_query() -> None:
    llm = Ollama(
        model="llama3.1",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        stop=["<|eot_id|>"],
    )

    query = input("\nQuery: ")
    qa_chain = RetrievalQA.from_chain_type(
        llm, retriever=vector_store.as_retriever(collection =  OAQ3_COLLECTION_NAME, expr="parameter == 'o3'"))

    result = qa_chain.invoke({"query": query})
    
    resultforslack = str(result["result"])
    print(resultforslack)

    try:
        response = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                            blocks=[{"type": "section","text": {"type": "mrkdwn","text": "*" + str(query) + "*  \n\n" + str(resultforslack) +"\n" }}])

    
    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        print("Slack failed")

In [ ]:
    while True:
        run_query()

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_39496/3344370668.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  Provide a detailed air quality report for Trenton


Based on the provided context, here is a detailed air quality report for Trenton:

**Current Air Quality Status**

The current air quality reading for ozone (o3) in Trenton is 0.06 ppm at 16:00:00-04:00 on September 10th, 2024.

**Recent Air Quality Readings**

Here are the recent air quality readings for o3 in Trenton:

* At 11:00:00-04:00 on September 10th, 2024, the reading was 0.045 ppm.
* At 09:00:00-04:00 on September 10th, 2024, the reading was 0.03 ppm.
* At 15:00:00-04:00 on September 10th, 2024, the reading was 0.061 ppm.

**Trends and Patterns**

There appears to be a slight increase in o3 levels throughout the day, with readings increasing from 0.03 ppm at 09:00:00-04:00 to 0.06 ppm at 16:00:00-04:00. However, it's worth noting that these changes are relatively small and may not be statistically significant.

**Sensor Information**

The air quality data is provided by a reference-grade sensor located at the specified coordinates (33.7397,-81.8536) in Trenton. The sensor is 

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_39496/3344370668.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  Provide a detailed air quality report for Houston


...

I'm not able to provide the specific air quality report you're asking for. I can, however, use the context provided to give a general air quality update for Houston as of 2024-09-11.

Based on the given readings:

* At 10:00 AM (2024-09-11T10:00:00-05:00), the PM25 level was 8.0 µg/m³.
* At 12:00 PM (2024-09-11T12:00:00-05:00), the PM25 level dropped to 5.0 µg/m³.
* There are two readings at 1:00 AM (2024-09-11T01:00:00-05:00) showing a consistent PM25 level of 7.0 µg/m³.

This data suggests that the air quality in Houston on September 11, 2024, fluctuated throughout the day but generally remained at a moderate level for PM25, with values ranging from 5 to 8 µg/m³. This is below the US Environmental Protection Agency's (EPA) standard threshold for unhealthy air quality.

For a detailed report that includes other pollutants and provides more specific information about health risks or recommendations, I recommend consulting the actual source of this data or reaching out to local air qual

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_39496/3344370668.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  Provide a detailed air quality report for Trenton including PM25


Based on the provided context, here is a detailed air quality report for Trenton:

**Location:** Trenton, None, US
**Coordinates:** 33.7397,-81.8536
**Sensor Type:** Reference grade
**Entity:** Governmental Organization
**Mobile:** False (i.e., this reading comes from a stationary sensor)

**Current Air Quality Readings:**

1. **Time:** 2024-09-10T06:00:00-04:00
	* PM25: 5.8 µg/m³
2. **Time:** 2024-09-10T09:00:00-04:00 (same reading as above)
	* PM25: 10.7 µg/m³

**Interpretation of Readings:**

The current air quality readings for Trenton indicate a moderate level of particulate matter (PM) in the air. The PM25 levels are:

* 5.8 µg/m³ at 06:00, which is relatively good air quality.
* 10.7 µg/m³ at 09:00, which indicates a slight increase in PM levels.

For context, the US Environmental Protection Agency (EPA) sets the following guidelines for PM25:
	+ Good Air Quality: ≤ 12.0 µg/m³
	+ Moderate Air Quality: > 12.0 µg/m³ and ≤ 35.4 µg/m³
	+ Unhealthy Air Quality: > 35.4 µg/m³

Based on

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_39496/3344370668.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(
